In [1]:
# activating the debugger
%pdb


Automatic pdb calling has been turned ON


In [3]:
# necessary backages
import unicodecsv


In [4]:
def read_csv(file_path):
    'takes a path of csv file and retuns list it converted to a list'
    with open(file_path, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)
enrollments =  read_csv('../../enrollments.csv')   
project_submissions =  read_csv('../../daily_engagement.csv')
daily_engagement =  read_csv('../../project_submissions.csv')

print project_submissions[0]
print daily_engagement[0]

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}
